In [ ]:
import pandas as pd
import os

La tabla que limita a las demás es la de las descripciones

In [ ]:
df_limitante = pd.read_excel('DescripcionesEspecies.xlsx')

#### Cargamos el resto de tablas que hacen referencia a especies, y eliminamos aquellas que no esten en la tabla anterior

Tabla de taxonomía

In [ ]:
df_taxonomia = pd.read_excel('EIDOS_taxonomia.xlsx')
df_taxonomia[df_taxonomia['taxonid'].isin(df_limitante['idtaxon'])][[
    'taxonid', 'name', 'taxonrank', 'origen', 'taxonomicgroup', 'kingdom',
       'phylum', 'class', 'order', 'family', 'genus', 'subgenus',
       'specificepithet', 'infraspecificepithet', 'nametype'
]]
df_taxonomia.to_csv('Taxonomia.csv', index=False)
os.remove('EIDOS_taxonomia.xlsx')

Tabla de ubicaciones (cuadrículas) de especies

In [ ]:
df_cuadriculas = pd.read_csv('UBICACIONES_TODAS.csv')
df_cuadriculas[df_cuadriculas['idtaxon'].isin(df_limitante['idtaxon'])]
df_cuadriculas.to_csv('CuadriculasEspecies.csv', index=False)
os.remove('UBICACIONES_TODAS.csv')

Tabla de nombres de especies

In [14]:
df_nombres = pd.read_excel('EIDOS_nombres.xlsx')
df_nombres[df_nombres['idtaxon'].isin(df_limitante['idtaxon'])]
df_nombres = df_nombres[['idtaxon', 'nombre_aceptado', 'nombre_comun', 'idioma', 'espreferente']]
df_nombres.to_csv('NombresEspecies.csv', index=False)
os.remove('EIDOS_nombres.xlsx')